In [ ]:
%cd ..

In [ ]:
import io
import json
from dataclasses import dataclass, field
from pathlib import Path
from xml.etree import ElementTree
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
def get_files(dirname: Path, extension: str):
    return {path.stem: path for path in dirname.glob(f"*.{extension}")}

In [ ]:
root = Path("/mnt/fast/data")

# Caption Codes

In [ ]:
from pytube import YouTube

In [ ]:
videos = json.loads(Path("data/videos.json").read_text())

In [ ]:
codes = {}
for key, url in videos.items():
    video = YouTube(url)    
    codes[key] = tuple(caption.code for caption in video.captions)

In [ ]:
keys = list(videos)

In [ ]:
Path("data/codes.json").write_text(json.dumps(codes))

# Reading JSON and XML Captions

In [ ]:
def read_xml(fn: Path):
    buf = io.StringIO(fn.read_text())
    tree = ElementTree.parse(buf).getroot()
    segments = []
    for segment in tree.findall("body/p"):
        start, delta = [int(segment.attrib[k]) for k in ("t", "d")]
        text = segment.text.replace("\n", " ").strip()
        segments.append({"start": start, "end": start+delta, "text": text})
    return segments

In [ ]:
def read_json(fn: Path):
    return [
        {
            k: segment[k] if k == "text" else int(segment[k])*1000 
            for k in ("start", "end", "text")
        } 
        for segment in json.loads(fn.read_text())
    ]

In [ ]:
def to_text(segments: list) -> str:
    return " ".join([x["text"].strip() for x in segments])

In [ ]:
@dataclass
class Transcript:
    title: str
    reference: list
    whisper: list
    
    def get_a(self, index): return self.reference[index]["text"]

    def get_b(self, index): return self.whisper[index]["text"]
    
    def to_text(self):
        return to_text(self.reference), to_text(self.whisper)
    
    def __repr__(self):
        return f"Transcript(title='{self.title}', n={len(self.reference)}, m={len(self.whisper)})"

@dataclass
class TranscriptStorage:
    root: Path
    subdirs: tuple[str, str]
    keys: list = field(init=False)
    parsed: list = field(init=False)
    manual: list = field(init=False)
    
    def __post_init__(self):
        parsed, manual = self.subdirs
        self.parsed = get_files(self.root/parsed, "json")
        self.manual = get_files(self.root/manual, "xml")
        self.keys = sorted(self.manual)
        
    def __len__(self): return len(self.keys)
    
    def get(self, item) -> Transcript:
        key = item if isinstance(item, str) else self.keys[item]
        ref = read_xml(self.manual[key])
        cleaned_key = "".join([c for c in key if c not in ":;,#"])
        new = read_json(self.parsed[cleaned_key])
        return Transcript(key, ref, new)

In [ ]:
model_names = "tiny_en", "base_en", "small_en", "medium_en"

In [ ]:
dfs = []

for name in tqdm(model_names):
    storage = TranscriptStorage(root, (f"lectures_parsed_{name}", "lectures"))
    n_total = len(storage)

    rows = []
    for i in range(n_total):
        tr = storage.get(i)
        ref, new = tr.to_text()
        rows.append({"title": tr.title, "ref": ref, name: new})
    
    dfs.append(pd.DataFrame(rows))

In [ ]:
def sort_key(series):
    
    def _key(x):
        from string import digits
        if x.startswith("Lesson"):
            lesson = int("".join([ch for ch in x.split()[1] if ch in digits]))
        else:
            lesson = 0
        lecture_year = 0
        for year in range(2018, 2023):
            if str(year) in x:
                lecture_year = year
        return lecture_year, lesson
        
    return series.map(_key)

In [ ]:
from functools import reduce
df = reduce(pd.merge, dfs).sort_values("title", key=sort_key)
df.head()

In [ ]:
df.to_csv(root/"lectures.csv", index=False)